In [1]:
import os 
import re
import glob 
import random

from tqdm import tqdm
import joblib
import pickle
from pymongo import MongoClient
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from numpy import array
from pymongo import MongoClient
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import datetime,timedelta
import warnings 
warnings.filterwarnings(action='ignore')

#### 1.) Postgre 학습데이터 구축
- 장비(maker) 1,2,3 Raw Data 학습용 데이터로 구축
- Lot단위 불량률 집계 중심 데이터 전처리
- Setting기준 데이터 군집 분리
- 장비별 Sensing(Production) & Setting 변수 명칭 List 정의/Numpy 형식 저장

In [161]:
#### 1.) Ex.) 장비1 생산 데이터 업로드 & 기초 전처리 적용
df1 = pd.read_csv('postgre_data/Maker1_Production_Postgre..csv',encoding='cp949')
df1.drop(['param_inject_start_pos'],axis=1,inplace=True)
df1['facility_cd'] = df1['facility_cd'].astype(str)
df1 = df1[df1['operation_mode']=='RUN'].reset_index(drop=True)
df1 = df1.drop_duplicates().reset_index(drop=True)
df_inputs = df1.loc[:, df1.columns.str.contains('shotdata_')]
unique_shot_cols = df_inputs.loc[:,df_inputs.apply(pd.Series.nunique) == 1].columns.tolist()
print(unique_shot_cols,' ==> Shot 변동 0회 제거 ')
df1 = df1.drop(unique_shot_cols,axis=1)
df1 = df1.dropna(subset=['item_cd']).reset_index(drop=True)
df1 = df1.dropna(axis=1).dropna().reset_index(drop=True)
df1.rename(columns=lambda x: x.replace('param_', 'set_'), inplace=True)


#### 2.) Ex.) 장비2 생산실적 데이터 업로드 & 기초 전처리 적용
df2 = pd.read_csv('postgre_data/Maker1_Result_Postgre.csv',encoding='cp949')
df2 = df2.dropna(subset=['cast_no','moldName','item_cd','ITEMNM']).reset_index(drop=True)
df2['facility_cd'] = df2['facility_cd'].astype(str)
df2['m_shotdata_no'] = df2['m_shotdata_no'].astype(int)
df2.rename(columns={'m_shotdatetime':'shotdatetime','m_shotdata_no':'shotdata_no'},inplace=True)
df2 = df2.drop_duplicates(subset=['facility_cd','item_cd','shotdata_no','shotdatetime']).reset_index(drop=True)
df2[['good_qty_sum','bad_qty_sum']] = df2.groupby(['lot_no','cast_no'])[['good_qty','bad_qty']].transform('sum')
df2['qty_sum'] = df2['good_qty_sum'] + df2['bad_qty_sum']
df2[['seq_sum']] = df2.groupby(['lot_no','cast_no'])[['seq']].transform('count')
df2['cavity_count'] = np.floor(df2['qty_sum']/df2['seq_sum'])
df2['qty_(cavity)_equals_or_large'] = df2['seq_sum'] <= df2['qty_sum']
df2 = df2[df2['qty_(cavity)_equals_or_large']==True].reset_index(drop=True)
df2 = df2.dropna(axis=1).dropna().reset_index(drop=True)

print(df1.shape, df2.shape)


#### 3.) 생산 & 실적 공통 Column 탐색 -->  [ 'plant_cd','facility_cd','shotdatetime','shotdata_no' ] Key 매칭 가능 
same_cols = df1.columns.intersection(df2.columns).tolist()
print(same_cols)


#### 4.) 생산 & 실적 Key Column 기반 데이터 매칭 (1차 완성)
df_merged = pd.merge(df1,df2,on=same_cols,how='inner')
df_merged = df_merged.drop_duplicates()
df_merged.shotdatetime = pd.to_datetime(df_merged.shotdatetime)


#### 5.) Good & Bad Qty 동시 0개 / Good & Bad Qty 같은 개수 삭제 ( 학습 혼동 방지 )
df_merged = df_merged[(df_merged['good_qty']!=0.0) | (df_merged['bad_qty']!=0.0)].reset_index(drop=True) # 생산 없음 
df_merged['qty_fault_percentage'] = df_merged['bad_qty'] /(df_merged['good_qty'] + df_merged['bad_qty'])


#### 6.) Cavity생산기준 기준 1차 라벨링 
df_merged['PassOrFail'] = [1 if x > 0.0 else 0 for x in df_merged['qty_fault_percentage']]


#### 7.) lot & cavity 불량률 총계산 --> ** 생산날짜 기준 총 생산 수량 중 불량비율 계산 ** 
df_merged[['good_qty_sum','bad_qty_sum']] = df_merged.groupby(['facility_cd','item_cd','lot_no','cast_no'])[['good_qty','bad_qty']].transform('sum')
df_merged['qty_sum'] = df_merged['good_qty_sum'] + df_merged['bad_qty_sum']
df_merged[['seq_sum']] = df_merged.groupby(['facility_cd','item_cd','lot_no','cast_no'])[['seq']].transform('count')
df_merged['cavity_count'] = np.floor(df_merged['qty_sum']/df_merged['seq_sum'])
df_merged['lot_fault_ratio'] =  (df_merged['bad_qty_sum']/(df_merged['qty_sum']))*100
df_merged['cavity_fault_ratio'] = (df_merged['bad_qty'] /(df_merged['good_qty'] + df_merged['bad_qty']))*100
df_merged = df_merged.dropna(axis=1).dropna(axis=0).reset_index(drop=True)
print(df_merged.shape)
df_merged.rename(columns={'facility_cd':'MECHCD','item_cd':'SABUN'},inplace=True)
df_summary = df_merged[['maker','moldNumber','moldName','MECHCD','MECHNM','SABUN','ITEMNM','shotdatetime','shotdata_no','lot_no','cast_no','seq','seq_sum','good_qty','bad_qty','qty_sum','cavity_count','lot_fault_ratio']]
display(df_summary.tail())


#### 8.) Setting변동 기준 [ Facility(설비) --> ITEM(품목) --> Setting구분(KMeans Cluster) ] 3단계 정보처리
set_cols = df_merged.loc[:,df_merged.columns.str.contains('set_')].columns 
for col in set_cols:
    try:
        df_merged[col] = df_merged[col].astype('float')
    except:
        df_merged[col] = df_merged[col].astype('float')
n = len(df_merged[set_cols].drop_duplicates()) # set_cols 
df_set = df_merged[set_cols]
pipe1 = Pipeline([('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=n, random_state=0))])
df_clustered_setting = pipe1.fit_predict(df_set)
df_merged['clusterSetting'] = df_clustered_setting
cluster_settings = df_merged['clusterSetting'].unique().tolist()
cluster_settings.sort()
joblib.dump(pipe1,'cluster_settings/Maker1_settings.pkl')
print('Setting Clusters ==> ',len(cluster_settings))


#### 9.) 생산 데이터에 대한 Setting변동 분할 이후 적은 데이터 개수 필터링 & 학습정보 Dictionary 형태 저장
grouped_counts = (df_merged.groupby(['MECHCD', 'SABUN', 'clusterSetting']).size().reset_index(name='count'))
grouped_counts['cluster_trainable'] = grouped_counts['count'] >= 500
grouped_counts.to_csv('dictionary/Maker1_dictionary.csv',encoding='cp949',index=False) 
df_merged = df_merged.merge(grouped_counts[['MECHCD','SABUN','clusterSetting', 'cluster_trainable']],
    on=['MECHCD', 'SABUN', 'clusterSetting'],how='inner')
print('Trainable Clusters ==> ',len(df_merged[df_merged['cluster_trainable']==True]['clusterSetting'].unique()))


#### 10.) 최종 학습용 데이터 장비별 저장 완료 
print(df_merged['lot_fault_ratio'].value_counts())
display(df_merged.groupby(['MECHCD', 'SABUN','lot_no','clusterSetting'])[['good_qty', 'bad_qty']].sum().tail(20))
df_merged.drop(['shotdata_no'],axis=1,inplace=True)
df_merged.to_csv('학습용데이터/Maker1_train.csv',encoding='cp949',index=False)


#### 11.) 장비별 Sensing(Production) & Setting Column 명칭 정의 & 정보 저장 
shot_cols_postrgre = [col for col in df_merged.columns if col.split('_')[0] == 'shotdata']    
set_cols_postgre = [col for col in df_merged.columns if col.split('_')[0] == 'set']
np.save(f"columns/Maker1/shot_cols.npy",np.array(shot_cols_postrgre))
np.save(f"columns/Maker1/setting_cols.npy",np.array(set_cols_postgre))

[]  ==> Shot 변동 0회 제거 
(929310, 89) (1059495, 46)
['plant_cd', 'facility_cd', 'shotdatetime', 'shotdata_no', 'item_cd']
(913442, 134)


,maker,moldNumber,moldName,MECHCD,MECHNM,SABUN,ITEMNM,shotdatetime,shotdata_no,lot_no,cast_no,seq,seq_sum,good_qty,bad_qty,qty_sum,cavity_count,lot_fault_ratio
913437,ENGEL,23M3830L,MQ4PE_OPTICAL LENS-LOW HIGH(1차),1015,1-4호기,W-CG921-37220-A,MQ4PE_OPTICAL LENS-LOW HIGH(1차),2025-05-15 13:06:12,596600,20250515,23M3830L,83129,256,4.0,0.0,1024.0,4.0,0.0
913438,ENGEL,23M3830L,MQ4PE_OPTICAL LENS-LOW HIGH(1차),1015,1-4호기,W-CG921-37220-A,MQ4PE_OPTICAL LENS-LOW HIGH(1차),2025-05-15 13:07:19,596601,20250515,23M3830L,83163,256,4.0,0.0,1024.0,4.0,0.0
913439,ENGEL,23M3830L,MQ4PE_OPTICAL LENS-LOW HIGH(1차),1015,1-4호기,W-CG921-37220-A,MQ4PE_OPTICAL LENS-LOW HIGH(1차),2025-05-15 13:08:23,596602,20250515,23M3830L,83213,256,4.0,0.0,1024.0,4.0,0.0
913440,ENGEL,23M3830L,MQ4PE_OPTICAL LENS-LOW HIGH(1차),1015,1-4호기,W-CG921-37220-A,MQ4PE_OPTICAL LENS-LOW HIGH(1차),2025-05-15 13:09:30,596603,20250515,23M3830L,83247,256,4.0,0.0,1024.0,4.0,0.0
913441,ENGEL,23M3830L,MQ4PE_OPTICAL LENS-LOW HIGH(1차),1015,1-4호기,W-CG921-37220-A,MQ4PE_OPTICAL LENS-LOW HIGH(1차),2025-05-15 13:10:36,596604,20250515,23M3830L,83301,256,4.0,0.0,1024.0,4.0,0.0


Setting Clusters ==>  527
Trainable Clusters ==>  164
lot_fault_ratio
0.000000     826967
0.452899       1104
0.091075       1098
0.388350       1030
0.501505        997
              ...  
2.272727         44
2.564103         39
3.225806         31
3.846154         26
10.000000        20
Name: count, Length: 167, dtype: int64


good_qty  bad_qty
MECHCD SABUN           lot_no   clusterSetting                   
M1006  W-N1921-37540-A 20250312 30                1920.0      0.0
                       20250313 30                 108.0      0.0
                       20250316 30                1612.0      0.0
                                131                  8.0      0.0
                       20250317 30                2444.0      0.0
                       20250318 30                2470.0      0.0
                       20250319 30                 200.0      0.0
                       20250320 30                1646.0      0.0
                       20250321 30                2244.0      0.0
                       20250324 30                2324.0      0.0
                       20250325 30                1697.0      0.0
                                187                756.0      0.0
                                457                  4.0      0.0
                       20250326 187               2410.0      0.0
                       20250327 187               2428.0      0.0
                       20250328 187               1200.0      0.0
                       20250402 168                 31.0      0.0
                                187                934.0      0.0
                       20250403 187               2324.0      0.0
                       20250404 187               1226.0      0.0